In [1]:
%load_ext sagemaker_studio_analytics_extension.magics
%sm_analytics emr connect --cluster-id j-1P3CAFFX0W7OJ --auth-type None  

Successfully read emr cluster(j-1P3CAFFX0W7OJ) details
Initiating EMR connection..
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
11,application_1654626007541_0012,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.
{"namespace": "sagemaker-analytics", "cluster_id": "j-1P3CAFFX0W7OJ", "error_message": null, "success": true, "service": "emr", "operation": "connect"}


<h1>Configuration</h1>

In [8]:
%%configure -f
{ "conf": {
     "spark.sql.legacy.allowUntypedScalaUDF": "true"
     },
  "jars": [
        "s3://sales-tim-mckenzie/com.pb.software/spectrum-bigdata/5.1.0.8/spectrum-bigdata-addressing-5.1.0.8/spark2/sdk/lib/spectrum-bigdata-addressing-sdk-spark2_2.12-5.1.0.8.jar",
        "s3://sales-tim-mckenzie/com.pb.software/spectrum-bigdata/5.1.0.8/custom/ScalaGeocodingUDF-scala_geocode_udf_1.0-all.jar"
      
    ]   
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
12,application_1654626007541_0013,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
12,application_1654626007541_0013,pyspark,idle,Link,Link,None,✔


In [9]:
from pyspark.sql import *
from pyspark.sql.functions import col
import os
import time
sqlContext = SQLContext(spark.sparkContext)

dataLocationLocal = ["s3://sales-tim-mckenzie/com.precisely.data/reference-data/2022.4/KLD042022.spd","s3://sales-tim-mckenzie/com.precisely.data/reference-data/2022.4/KGD042022.spd"]

geocodingRootDBFS = "s3://sales-tim-mckenzie/com.pb.software/spectrum-bigdata/5.1.0.8/spectrum-bigdata-addressing-5.1.0.8/"
resourcesLocationDBFS = geocodingRootDBFS + "resources/"
resourcesLocationLocal = resourcesLocationDBFS

InputFile_Loaction = "s3://sales-tim-mckenzie/Customer/ClearCapital/2022/implementation/amv_test_file_match-05_25_2022.csv"



outputFields = ["score",
           "explanation.source['label'] as label",
           "location.feature.geometry.coordinates.x as LON",
           "location.feature.geometry.coordinates.y as LAT",
           "address.formattedAddress as FullAddress",
           "address.formattedStreetAddress as StreetAddress",
           "address.formattedLocationAddress as LocationAddress",
           "address.placeName as PlaceName",
           "address.addressNumber as AddressNumber",
           "address.street as Street",
           "address.unit as Unit",
           "address.unitType as UnitType",
           "address.admin1.longName as State",
           "address.city.longName as City",
           "address.postalCode as PostalCode",
           "address.postalCodeExt as PostalCodeExt",
           "address.country.name as CountryName",
           "address.country.isoAlpha3Code as Country",
           "customFields['PRECISION_CODE'] as PrecisionCode",
           "customFields['MATCH_TYPE'] as MatchType",
           "customFields['LOC_CODE'] as LocationCode",
           "customFields['MATCH_CODE'] as MatchCode",
           "customFields['PB_KEY'] as PBKEY",
           "customFields['DPV_CONFIRM'] as DpvConfirm",
           "customFields['Geocode_Pass'] as GeocodePass"]



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt/yarn/usercache/livy/appcache/application_1654626007541_0013/container_1654626007541_0013_01_000001/pyspark.zip/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.

<h1>Input File</h1>

In [10]:
inputDF = spark.read.option("header", "true").option("delimiter", ",").format("csv").load(InputFile_Loaction)
old_col=inputDF.schema.names
running_list=[]
new_col=[]
prefix='input'
for column in old_col:
    if(column not in running_list):
        new_col.append(prefix + "_" + column)
    else:
        new_col.append(column)
        running_list.append(column)

inputDF=inputDF.toDF(*new_col)
inputDF.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+--------------+----------------+------------+--------------+-----------+-----------+-----------+---------+--------------------+--------------------+---------------------+---------------+-------------------+---------------+----------+--------------+-------------+----------------+----------------+-------------------+-----------------+-------------+-------------------+---------------+------------------+---------------+------------+-----------+
|input__c0|      input_address1|input_address2|     input_city3|input_state4|input_postcode|input_score|input_label|  input_LON|input_LAT|   input_FullAddress| input_StreetAddress|input_LocationAddress|input_PlaceName|input_AddressNumber|   input_Street|input_Unit|input_UnitType|input_State18|    input_City19|input_PostalCode|input_PostalCodeExt|input_CountryName|input_Country|input_PrecisionCode|input_MatchType|input_LocationCode|input_MatchCode| input_PBKEY|input_error|
+---------+--------------------+--------------

<h1>Scala Python UDF</h1>

In [11]:
downloadLocationLocal = "/mnt/precisely/"
inputFields = {'addressLines[0]' : '1,2', 'city' : '3', 'admin1' : '4', 'postalCode' : '5'}
finalDF = DataFrame(sqlContext._jvm.com.precisely.Addressing.addressingDF(inputDF._jdf, spark.sparkContext._jsc,  "geocode", resourcesLocationLocal, downloadLocationLocal, dataLocationLocal, outputFields, inputFields), sqlContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h1>Output</h1>

In [12]:
finalDF.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+--------------+----------------+------------+--------------+-----------+-----------+-----------+---------+--------------------+--------------------+---------------------+---------------+-------------------+---------------+----------+--------------+-------------+----------------+----------------+-------------------+-----------------+-------------+-------------------+---------------+------------------+---------------+------------+-----------+-----+-----+-----------+---------+--------------------+--------------------+--------------------+---------+-------------+---------------+----+--------+-------------+----------------+----------+-------------+-------------+-------+-------------+---------+------------+---------+------------+----------+-----------+-----+
|input__c0|      input_address1|input_address2|     input_city3|input_state4|input_postcode|input_score|input_label|  input_LON|input_LAT|   input_FullAddress| input_StreetAddress|input_LocationAddress|inp